##  감성체인 생성

In [8]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [9]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [10]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名中国社会科学家，你的任务是分析关于“美日韩”三边关系的中国新闻报道，进行情感分析。
    以下是关于“美日韩”三边关系的中国新闻报道中的一段话，请对每个句子进行情感分析

      **评分标准**：
      - 强烈负面 (-5): 极端消极
      - 负面 (-3 ~ -4): 较为消极
      - 轻微负面 (-1 ~ -2): 略微消极
      - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
      - 轻微正面 (+1 ~ +2): 略微积极
      - 正面 (+3 ~ +4): 较为积极
      - 强烈正面 (+5): 极端积极

      **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。所有提供的句子都必须进行情感分析，不得遗漏任何句子**。
      **新闻文章内容:**
      {sentences}

      **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
      """
)

In [11]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [12]:
import json

# JSON 파일 로드
file_path = "../data/0306/smaple.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 6개의 기사 로드 완료


## 감성분석 수행 함수

In [13]:
# 감성 분석 수행 함수
def analyze_sentiment_per_sentence(article):
    """기사의 각 문장에 대해 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    sentiment_scores = []
    try:
        # 문장별로 감성 분석 수행
        for sentence in article["sentences"]:
            input_data = {"sentences": sentence}  # 각 문장 하나씩 전달

            # `sentiment_chain.run()`을 사용하여 데이터 처리
            response = sentiment_chain.invoke(input_data)  # 문장별 감성 분석 요청

            print(f"Response format: {response}")  # 응답 형식 출력

            # 응답에서 각 문장에 대한 감성 점수 저장
            sentiment_scores.append(response['text'].strip())  # 응답 내용 반환
        article["sentiment_responses"] = sentiment_scores  # 문장별 감성 점수 저장
        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response:
        try:
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment_per_sentence, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article is not None:
                results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")


In [14]:
output_path = "../data/0306/results문장하나하나.json"
run_sentiment_analysis(data, output_path)

2025-03-06 14:18:15,406 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-06 14:18:15,579 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': '环球网记者王欢报道 10月11日上午，正在韩国访问的日本民主党政调会长前原诚司在首尔发表演讲。', 'text': '0'}
Response format: {'sentences': '朝鲜国防委员会１６日提议与美国举行政府间高级别会谈之后，美国、日本和韩国分别回应提议。', 'text': '0\n0'}


2025-03-06 14:18:16,076 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': '据共同社6月21日报道，日本海上自卫队同美韩两国海军的首次联合军演21日在朝鲜半岛以南的公海拉开帷幕。', 'text': '0'}


2025-03-06 14:18:16,385 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-06 14:18:16,524 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2025-03-06 14:18:49,966 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': '日本政府当然不认同这种看法。', 'text': '0'}


2025-03-06 14:18:50,753 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


KeyboardInterrupt: 

Response format: {'sentences': '安倍不应纠结于这个问题。', 'text': '-2'}
